In [1]:
import os
import csv
import numpy as np
from dotenv import load_dotenv
from tqdm.notebook import tqdm

from huggingface_hub import login
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import CosineAnnealingLR

from pricer.evaluator import evaluate
from pricer.items import Item

In [2]:
# Static configuration

LITE_MODEL = False
EPOCHS = 30
BATCH_SIZE = 64
LEARNING_RATE = 0.001
WEIGHT_DECAY = 1e-4
N_FEATURES = 5000
VAL_SPLIT = 0.1
PATIENCE = 5
RANDOM_SEED = 42

In [3]:
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [4]:
load_dotenv(override=True)
hf_token = os.environ["HF_TOKEN"]
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODEL else f"{username}/items_full"

In [6]:
# Load dataset

train, val, test = Item.from_hub(dataset)
print(f"Loaded {len(train):,} training items, {len(val):,} validation_items, {len(test):,} test items")

Loaded 800,000 training items, 10,000 validation_items, 10,000 test items


In [7]:
# Human baseline from previous testing

human_predictions = []
with open("human_in.csv", "w", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:100]:
       writer.writerow([t.summary, 0])

In [8]:
human_predictions = []
with open("human_out.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[1]))

In [9]:
def human_pricer(item):
    idx =test.index(item)
    return human_predictions[idx]

In [10]:
human = human_pricer(test[0])
actual = test[0].price
print(f"Human predicted {human} for an item that actually cost: {actual}")

Human predicted 120.0 for an item that actually cost: 219.0


In [11]:
evaluate(human_pricer, test, size=100)

  0%|          | 0/100 [00:00<?, ?it/s]

$99 $184 $12 $15 $18 $10 $119 $135 $6 $270 $643 $329 $15 $26 $24 $18 $29 $25 $25 $53 $35 $126 $25 $127 $273 $398 $55 $6 $101 $51 $30 $5 $35 $9 $10 $419 $25 $11 $186 $33 $161 $51 $23 $155 $150 $4 $31 $18 $115 $82 $25 $111 $410 $75 $67 $34 $8 $10 $122 $28 $116 $17 $19 $60 $599 $60 $160 $355 $75 $34 $17 $2 $70 $76 $41 $9 $226 $5 $5 $4 $0 $7 $5 $74 $7 $10 $68 $74 $5 $3 $17 $45 $5 $16 $0 $153 $2 $122 $150 $355 

In [12]:
# Log transform prices to normalize the skewed distribution

y_raw = np.array([float(item.price) for item in train])
y = np.log1p(y_raw)

In [13]:
print(f"Raw prices - Mean: {y_raw.mean():.2f}, Median: {np.median(y_raw):.2f}, Std: {y_raw.std():.2f}")
print(f"Log prices - Mean: {y.mean():.2f}, Median: {np.median(y):.2f}, Std: {y.std():.2f}")

Raw prices - Mean: 140.57, Median: 80.10, Std: 160.07
Log prices - Mean: 4.43, Median: 4.40, Std: 1.03


In [14]:
# Bag of words vectorization with hashing (memory efficient, no vocab storage needed)

documents = [item.summary for item in train]
vectorizer = HashingVectorizer(n_features=N_FEATURES, stop_words="english", binary=True)
X = vectorizer.fit_transform(documents)

In [15]:
print(f"Features matrix shape: {X.shape}")

Features matrix shape: (800000, 5000)


In [16]:
# NN architecture

class ImprovedPriceNueralNetwork(nn.Module):
    """
    3 hidden layer network with Batchnorm and Dropout.
    Wider first layer (256) to handle 5000 dimensional sparse input
    and then progressively narrow.
    """

    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(

            # Layer 1: Compress 5000 sparse features into dense representation
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),

            # Layer 2: Refine representation
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.1),

            # Layer 3: Final hidden layer
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.1),

            # Output: Single price prediction (in log space)
            nn.Linear(64, 1),
        )

    def forward(self, x):
        return self.net(x)


In [17]:
# Convert to Pytorch tensors

X_tensor = torch.FloatTensor(X.toarray())
y_tensor = torch.FloatTensor(y).unsqueeze(1)

In [18]:
# Train / validation split (val 10% per VAL_SPLIT config value)

X_train, X_val, y_train, y_val = train_test_split(
    X_tensor, y_tensor, test_size=VAL_SPLIT, random_state=RANDOM_SEED
)

print(f"Training samples: {X_train.shape[0]:.2f}")
print(f"Validation samples: {X_val.shape[0]:.2f}")

Training samples: 720000.00
Validation samples: 80000.00


In [19]:
# Dataloader for mini-back training

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [20]:
# Initialize model

input_size = X_tensor.shape[1]
model = ImprovedPriceNueralNetwork(input_size)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Traininable parameters: {trainable_params:,}")

Traininable parameters: 1,322,369


In [21]:
# Measure loss using HuberLoss - ore robust to outliers than MSE

loss_function = nn.HuberLoss(delta=1.0)

In [22]:
# Adam optimizer with weight decay for layer 2 regularization

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

In [23]:
# Cosine annealing to decay learning rate smoothly to zero

scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [24]:
# Training loop with eary stopping

best_val_loss = float("inf")
patience_counter = 0
train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    # Training phase
    model.train()
    epoch_losses = []

    for batch_X, batch_y in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{EPOCHS}"):
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())

    avg_train_loss = np.mean(epoch_losses)
    train_losses.append(avg_train_loss)

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = loss_function(val_outputs, y_val).item()
    val_losses.append(val_loss)

    # Step the learning rate schedule
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]

    print(
        f"Epoch [{epoch + 1}/EPOCHS] "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} | "
        f"LR: {current_lr}:.6f"
    )

    # Early stopping check
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")
        patience_counter = 0
        print(f"New best model saved (val_loss: {val_loss}:.4f)")
    else:
        patience_counter += 1
        print(f"No improvement ({patience_counter}/{PATIENCE})")
        if patience_counter >= PATIENCE:
            print(f"\nEarly stopping triggered at epoch {epoch + 1}")
            break

Epoch 1/30:   0%|          | 0/11250 [00:00<?, ?it/s]

Epoch [1/EPOCHS] Train Loss: 0.2807 | Val Loss: 0.2011 | LR: 0.0009972609476841367:.6f
New best model saved (val_loss: 0.20110392570495605:.4f)


Epoch 2/30:   0%|          | 0/11250 [00:00<?, ?it/s]

Epoch [2/EPOCHS] Train Loss: 0.2219 | Val Loss: 0.1945 | LR: 0.0009890738003669028:.6f
New best model saved (val_loss: 0.19454076886177063:.4f)


Epoch 3/30:   0%|          | 0/11250 [00:00<?, ?it/s]

...

Epoch [28/EPOCHS] Train Loss: 0.1170 | Val Loss: 0.1571 | LR: 1.092619963309716e-05:.6f
No improvement (2/5)


Epoch 29/30:   0%|          | 0/11250 [00:00<?, ?it/s]

Epoch [29/EPOCHS] Train Loss: 0.1140 | Val Loss: 0.1569 | LR: 2.7390523158633003e-06:.6f
New best model saved (val_loss: 0.1568501740694046:.4f)


Epoch 30/30:   0%|          | 0/11250 [00:00<?, ?it/s]

Epoch [30/EPOCHS] Train Loss: 0.1122 | Val Loss: 0.1570 | LR: 0.0:.6f
No improvement (1/5)


In [26]:
# Load the best model weights

model.load_state_dict(torch.load("best_model.pt"))
print(f"\nLoaded best model with val_los: {best_val_loss:.4f}")


Loaded best model with val_los: 0.1569


In [27]:
# Evaluation

def neural_network(item):
    """Predict price for a single item.  Returns price in original dollar scale."""
    model.eval()
    with torch.no_grad():
        vector = vectorizer.transform([item.summary])
        vector = torch.FloatTensor(vector.toarray())
        log_prediction = model(vector)[0].item()
    # Reverse the log1p transform and clamp to non negative
    return max(0, np.expm1(log_prediction))

In [28]:
# test

sample = test[0]
predicted = neural_network(sample)
print(f"Predicted: ${predicted:.2f} | Actual: ${sample.price:.2f}")

Predicted: $91.70 | Actual: $219.00


In [29]:
evaluate(neural_network, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$127 $76 $17 $61 $99 $123 $48 $10 $9 $103 $257 $128 $35 $109 $26 $5 $11 $15 $13 $32 $43 $35 $43 $140 $92 $239 $260 $6 $63 $53 $38 $16 $20 $47 $4 $253 $21 $20 $102 $2 $96 $22 $9 $16 $116 $10 $3 $2 $81 $13 $1 $80 $268 $75 $28 $76 $17 $41 $10 $9 $142 $42 $30 $20 $238 $27 $28 $286 $20 $115 $15 $7 $19 $77 $22 $11 $121 $6 $17 $3 $45 $22 $50 $69 $9 $115 $21 $84 $42 $50 $0 $13 $2 $1 $9 $119 $5 $4 $73 $187 $12 $34 $3 $39 $25 $4 $16 $262 $6 $130 $25 $40 $21 $45 $121 $37 $23 $51 $51 $122 $7 $65 $26 $42 $93 $35 $1 $3 $92 $6 $20 $19 $10 $1 $55 $8 $113 $36 $0 $14 $1 $25 $3 $74 $14 $7 $11 $291 $23 $13 $4 $21 $9 $27 $10 $98 $67 $8 $78 $2 $1 $10 $4 $4 $286 $9 $87 $21 $0 $16 $19 $4 $303 $24 $50 $68 $19 $11 $38 $57 $35 $7 $128 $11 $8 $26 $50 $138 $2 $20 $54 $4 $4 $179 $1 $18 $105 $9 $27 $10 